In [35]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')
from sublimpy import tidy, utils
import pytz
import datetime as dt

seconds_in_timestep = 60*30
from metpy.constants import density_water

from metpy.units import units
import pint_pandas
from metpy import constants
from scipy import interpolate
np.set_printoptions(suppress=True,precision=10)

: 

# Parameters for calculations

In [2]:
heights = [1,3,10]
HORIZ_GRID_SPACING = 50
VERT_GRID_SPACING = 20
start_date = '20221130'
end_date = '20230509'
    # streamwise-coords
# tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet")
    # slope-adjusted earthwise-coords
tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet")
    # sonic coords
# tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet")

# method_numerical_advection = 'wind_divergence'      # q * d/dx_i (u_i)
# method_numerical_advection = 'scalar_divergence'    # u_i * d/dx_i (q)
# method_numerical_advection = 'summed'               # q * d/dx_i (u_i) + u_i * d/dx_i (q)
# method_numerical_advection = 'derivative'           # d/dx_i (q * u_i)

# Prepare data

## Open SOS Measurement Dataset

In [3]:
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

## Add combined blowing snow flux variable
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue', 'snow flux', 1, 'ue',
)

## Add absolute humidity measurements by converting hygrometer measurements

In [4]:
tower_height_keys = tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height']).indices.keys()
for t, h in tower_height_keys:
    this_tower_height_tidy_df = tidy_df.query(
            f"tower == '{t}'"
        ).query(
            f"height == {h}"
        )
    specific_humidity_values = this_tower_height_tidy_df.query(
            "measurement == 'specific humidity'"
        ).set_index('time')[['value']].rename(columns={'value': 'specific humidity'})
    air_density_values = this_tower_height_tidy_df.query(
            "measurement == 'air density'"
        ).set_index('time')[['value']].rename(columns={'value': 'air density'})
    combined_df = specific_humidity_values.join(air_density_values)

    abs_humidity_values = (
        combined_df['specific humidity'].values * units('g/g')
    ).to('g/kg') * (
        combined_df['air density'].values * units('kg/m^3')
    ).m

    tidy_df = tidy.tidy_df_add_variable(
        tidy_df,
        abs_humidity_values,
        f"absolutehumidity_{int(h)}m_{t}",
        'absolute humidity',
        int(h),
        t
    )

## Calibrate gas analyzer measurements

We calibrate by assuming that all gas analyzers have the same seasonal mean as the corresponding hygrometer measurement on the central tower (at a given height)

In [5]:
hygrometer_absolute_humidity_mean = (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
).reset_index().query("tower == 'c'")

In [6]:
ec_absolute_humidity_mean = tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()

In [7]:
corrections_df = ec_absolute_humidity_mean.merge(
    hygrometer_absolute_humidity_mean[['height', 'value']].rename(columns={'value': 'truth'}),
    on='height'
)
corrections_df['offset'] = corrections_df['value'] - corrections_df['truth']
corrections_df

,variable,tower,height,value,truth,offset
0,h2o_10m_c,c,10.0,2.128948,2.080217,0.048731
1,h2o_10m_d,d,10.0,1.264312,2.080217,-0.815905
2,h2o_10m_ue,ue,10.0,1.091568,2.080217,-0.988648
3,h2o_10m_uw,uw,10.0,1.608586,2.080217,-0.471630
4,h2o_15m_c,c,15.0,1.954592,2.065882,-0.111290
5,h2o_1m_c,c,1.0,2.264087,2.113118,0.150968
6,h2o_1m_d,d,1.0,1.700445,2.113118,-0.412673
7,h2o_1m_ue,ue,1.0,1.310203,2.113118,-0.802916
8,h2o_1m_uw,uw,1.0,1.502313,2.113118,-0.610806
9,h2o_20m_c,c,20.0,0.934247,2.056363,-1.122116


Update dataset with corrections

In [8]:
src = tidy_df[tidy_df.measurement=='Water vapor density']
src = src[src.height.isin([1,3,10])]
src

,time,variable,value,height,tower,measurement
58,2022-11-30 00:00:00,h2o_10m_uw,0.537592,10.0,uw,Water vapor density
110,2022-11-30 00:00:00,h2o_1m_c,0.786468,1.0,c,Water vapor density
168,2022-11-30 00:00:00,h2o_3m_uw,0.724031,3.0,uw,Water vapor density
172,2022-11-30 00:00:00,h2o_10m_ue,0.015322,10.0,ue,Water vapor density
219,2022-11-30 00:00:00,h2o_1m_ue,0.010025,1.0,ue,Water vapor density
...,...,...,...,...,...,...
6502449,2023-05-09 17:30:00,h2o_1m_d,NaN,1.0,d,Water vapor density
6502476,2023-05-09 17:30:00,h2o_3m_d,2.589736,3.0,d,Water vapor density
6502575,2023-05-09 17:30:00,h2o_1m_uw,NaN,1.0,uw,Water vapor density
6502797,2023-05-09 17:30:00,h2o_10m_c,2.466102,10.0,c,Water vapor density


In [9]:
alt.Chart(
    (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
    ).reset_index()
).mark_point(shape='square', filled=True, color='black', size=20).encode(
    alt.X("value:Q"),
    alt.Y("height:Q")
).properties(width=150, height = 150)\
+ alt.Chart(
    tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
).mark_circle(size=40).encode(
    alt.X("value:Q"),
    alt.Y("height:Q"),
    alt.Color('tower:N')
).properties(width=150, height = 150)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [10]:


alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
    alt.Color("height:N"),
    detail='variable'
)

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1085128452.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src[ src.time > '20221212' ][ src.time < '20221214' ]


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [11]:
for idx, row in corrections_df.iterrows():
    src = tidy_df.query(f"variable == '{row['variable']}'")
    src = src.assign(value = src.value - row['offset'])
    tidy_df = tidy_df[tidy_df.variable != row['variable']]
    tidy_df = pd.concat([tidy_df, src])

In [12]:
alt.Chart(
    (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
    ).reset_index()
).mark_point(shape='square', filled=True, color='black', size=20).encode(
    alt.X("value:Q"),
    alt.Y("height:Q")
).properties(width=150, height = 150)\
+ alt.Chart(
    tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
).mark_circle(size=40).encode(
    alt.X("value:Q"),
    alt.Y("height:Q"),
    alt.Color('tower:N')
).properties(width=150, height = 150)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [13]:
src = tidy_df[tidy_df.measurement=='Water vapor density']
src = src[src.height.isin([1,3,10])]
abs_hum = alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line(strokeWidth=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Absolute humidity (g/m^3)").scale(zero=False),
    alt.Color("height:N"),
    alt.Shape('tower:N'),
    detail='variable'
).properties(width=600)

src = tidy_df[tidy_df.measurement=='snow depth']
snowdepth = alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line(strokeWidth=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Snow depth (m)"),
    alt.Shape('tower:N'),
    detail='variable'
).properties(width=600, height=150)

(snowdepth & abs_hum).resolve_scale(color='independent', shape='independent')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/427924892.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src[ src.time > '20221212' ][ src.time < '20221214' ]
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/427924892.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  src[ src.time > '20221212' ][ src.time < '20221214' ]


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


# Identify categories for timestamps

In [14]:
# Identify lists of timestamps for different categories
bs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value > 0").time
nobs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value == 0").time

decoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value < 0.43").time
weaklycoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value >= 0.43").query("value <= 0.61").time
coupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value > 0.61").time

ri_stable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

december_times = tidy_df[tidy_df.time.dt.month == 12].time
january_times = tidy_df[tidy_df.time.dt.month == 1].time
february_times = tidy_df[tidy_df.time.dt.month == 2].time
march_times = tidy_df[tidy_df.time.dt.month == 3].time
april_times = tidy_df[tidy_df.time.dt.month == 4].time

midwinter_times = tidy_df[tidy_df.time < '20230320'].time
spring_times = tidy_df[tidy_df.time > '20230320'].time

In [15]:
precip_df = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/precip_danny/precipitation_rate_gts_w23.nc")['corrected_prcp_rate_m2'].to_dataframe()

is_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series() + dt.timedelta(minutes=30)
])
    
is_not_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series() + dt.timedelta(minutes=30)
])

# Create tables

## Instrument location info (georeferenced)
We use a file with theodolite/GPS readings provided by NCAR. 

In [16]:
instrument_loc_df = pd.read_csv("~/Development/data/sublimationofsnow/SOSm.txt", names = ['ec', 'x', 'y', 'z'])
instrument_loc_df = instrument_loc_df[ 
    instrument_loc_df['ec'].str.startswith('CS')
    |
    instrument_loc_df['ec'].str.startswith('DS') 
    |
    instrument_loc_df['ec'].str.startswith('UWS') 
    |
    instrument_loc_df['ec'].str.startswith('UES') 
]
instrument_loc_df = instrument_loc_df[ 
    instrument_loc_df['ec'].str.endswith('T') 
    |
    instrument_loc_df['ec'].str.endswith('B') 
]
instrument_loc_df['top or bottom'] = instrument_loc_df['ec'].str[-1]
instrument_loc_df['tower'] = instrument_loc_df['ec'].apply(lambda str: str.split('S')[0].lower())
instrument_loc_df['height'] = instrument_loc_df['ec'].apply(lambda str: int(str.split('S')[1][:-1]))
instrument_loc_df = instrument_loc_df.drop(columns='ec')
instrument_loc_df = instrument_loc_df.pivot(index=['height', 'tower'], columns='top or bottom').reset_index()
instrument_loc_df = instrument_loc_df.set_index(['height', 'tower']).groupby(level=0, axis=1).mean()
instrument_loc_df

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/3668931392.py:21: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  instrument_loc_df = instrument_loc_df.set_index(['height', 'tower']).groupby(level=0, axis=1).mean()


x             y           z
height tower                                        
1      c      329002.47110  4.312171e+06  2861.36010
       d      329016.77615  4.312158e+06  2860.09135
       ue     329005.76010  4.312190e+06  2862.19275
       uw     328983.46715  4.312165e+06  2860.54055
2      c      329002.49025  4.312171e+06  2862.28475
3      c      329002.53920  4.312171e+06  2863.30060
       d      329016.82015  4.312158e+06  2862.02455
       ue     329005.80410  4.312190e+06  2864.12270
       uw     328983.52075  4.312165e+06  2862.50290
5      c      329002.53790  4.312171e+06  2865.33380
10     c      329002.65185  4.312171e+06  2870.36580
       d      329016.90980  4.312158e+06  2868.59560
       ue     329005.87765  4.312190e+06  2870.70875
       uw     328983.59905  4.312165e+06  2869.09490
15     c      329002.72120  4.312171e+06  2875.37755
20     c      329002.80045  4.312171e+06  2880.25445

## Wind field measurements

In [17]:
wind_field_df = tidy_df[tidy_df.measurement.isin(['u','v','w']) & tidy_df.height.isin(heights)]
wind_field_df = round(wind_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
wind_field_df

height                 1.0                                                   \
tower                     c                       d                      ue   
measurement               u       v       w       u       v       w       u   
time                                                                          
2022-11-30 00:00:00  0.8442 -2.0019 -0.0388  0.8241 -1.8621 -0.0258  0.7621   
2022-11-30 00:30:00 -0.2592 -1.0267 -0.0255 -0.1717 -1.0263 -0.0297 -0.3245   
2022-11-30 01:00:00  0.2995 -1.1331 -0.0291  0.3493 -1.0744 -0.0199  0.2869   
2022-11-30 01:30:00  0.6549 -1.6004 -0.0338  0.6431 -1.4701 -0.0231  0.6043   
2022-11-30 02:00:00 -0.2904 -0.8675 -0.0303 -0.2469 -0.8933 -0.0251 -0.3630   
...                     ...     ...     ...     ...     ...     ...     ...   
2023-05-09 15:30:00  0.4570 -2.2166 -0.0320     NaN     NaN     NaN     NaN   
2023-05-09 16:00:00  0.4924 -3.0680 -0.0242     NaN     NaN     NaN     NaN   
2023-05-09 16:30:00  0.6614 -1.8370 -0.0195     NaN     NaN     NaN     NaN   
2023-05-09 17:00:00  0.7511 -2.5587 -0.0278     NaN     NaN     NaN     NaN   
2023-05-09 17:30:00  0.6576 -2.1972 -0.0169     NaN     NaN     NaN     NaN   

height                                       ...    10.0                  \
tower                                    uw  ...       c       d           
measurement               v       w       u  ...       w       u       v   
time                                         ...                           
2022-11-30 00:00:00 -1.9373 -0.0491  0.3113  ... -0.0823  2.5216 -2.1892   
2022-11-30 00:30:00 -1.0934 -0.0513 -0.2333  ... -0.0342 -0.0728 -0.8173   
2022-11-30 01:00:00 -1.1269 -0.0483  0.0885  ... -0.0566  0.6077 -0.6620   
2022-11-30 01:30:00 -1.5538 -0.0509  0.1304  ... -0.0922  1.8981 -1.5050   
2022-11-30 02:00:00 -0.9753 -0.0559 -0.2954  ... -0.0281 -0.0230 -0.5847   
...                     ...     ...     ...  ...     ...     ...     ...   
2023-05-09 15:30:00     NaN     NaN     NaN  ... -0.0880  1.4622 -2.9177   
2023-05-09 16:00:00     NaN     NaN     NaN  ... -0.1084  1.9826 -4.2441   
2023-05-09 16:30:00     NaN     NaN     NaN  ... -0.0782  1.8170 -2.2859   
2023-05-09 17:00:00     NaN     NaN     NaN  ... -0.1521  2.2542 -3.1893   
2023-05-09 17:30:00     NaN     NaN     NaN  ... -0.1077  2.5035 -2.6379   

height                                                                       
tower                            ue                      uw                  
measurement               w       u       v       w       u       v       w  
time                                                                         
2022-11-30 00:00:00 -0.0065  2.2063 -2.3716  0.0385  1.9062 -2.7111 -0.0001  
2022-11-30 00:30:00  0.0189 -0.2529 -0.7708  0.0288 -0.2268 -0.7774 -0.0121  
2022-11-30 01:00:00  0.0028  0.4626 -0.7354  0.0229  0.4762 -0.8680 -0.0210  
2022-11-30 01:30:00 -0.0322  1.7291 -1.6854 -0.0059  1.4609 -2.0032 -0.0751  
2022-11-30 02:00:00  0.0230 -0.1797 -0.4797  0.0546 -0.0708 -0.5148 -0.0342  
...                     ...     ...     ...     ...     ...     ...     ...  
2023-05-09 15:30:00  0.0055  1.0546 -2.9264 -0.0150  0.7188 -3.2364 -0.0674  
2023-05-09 16:00:00 -0.0841  1.5315 -4.1925 -0.0776  0.8299 -4.6255 -0.0672  
2023-05-09 16:30:00  0.0087  1.5308 -2.4116 -0.0206  1.2450 -2.6569 -0.0639  
2023-05-09 17:00:00 -0.0677  2.0220 -3.2202 -0.0637  1.2884 -3.6343 -0.0591  
2023-05-09 17:30:00 -0.0062  2.2296 -2.8267 -0.0377  1.7479 -3.1988 -0.0505  

[7485 rows x 36 columns]

## Turbulent water vapor flux measurements

In [18]:
turb_flux_field_df = tidy_df[tidy_df.measurement.isin(['u_h2o_','v_h2o_','w_h2o_']) & tidy_df.height.isin(heights)]
turb_flux_field_df = round(turb_flux_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
turb_flux_field_df

height                 1.0                                                   \
tower                     c                       d                      ue   
measurement          u_h2o_  v_h2o_  w_h2o_  u_h2o_  v_h2o_  w_h2o_  u_h2o_   
time                                                                          
2022-11-30 00:00:00 -0.0023  0.0060 -0.0008 -0.0013  0.0067 -0.0008  0.0005   
2022-11-30 00:30:00 -0.0122  0.0142 -0.0007 -0.0032  0.0118 -0.0005 -0.0000   
2022-11-30 01:00:00 -0.0024  0.0032 -0.0003  0.0033  0.0017 -0.0001 -0.0000   
2022-11-30 01:30:00 -0.0007 -0.0005 -0.0004  0.0026 -0.0012 -0.0003  0.0004   
2022-11-30 02:00:00 -0.0013  0.0059 -0.0004 -0.0003  0.0132 -0.0001  0.0005   
...                     ...     ...     ...     ...     ...     ...     ...   
2023-05-09 15:30:00  0.0979 -0.0804  0.0118  0.0000  0.0000  0.0000  0.0000   
2023-05-09 16:00:00  0.0661 -0.0200  0.0150  0.0000  0.0000  0.0000  0.0000   
2023-05-09 16:30:00 -0.0115 -0.0110  0.0067  0.0000  0.0000  0.0000  0.0000   
2023-05-09 17:00:00  0.0438 -0.0504  0.0112  0.0000  0.0000  0.0000  0.0000   
2023-05-09 17:30:00 -0.0758 -0.0639  0.0072  0.0000  0.0000  0.0000  0.0000   

height                                       ...    10.0                  \
tower                                    uw  ...       c       d           
measurement          v_h2o_  w_h2o_  u_h2o_  ...  w_h2o_  u_h2o_  v_h2o_   
time                                         ...                           
2022-11-30 00:00:00 -0.0009  0.0000  0.0082  ...  0.0011 -0.0254  0.0075   
2022-11-30 00:30:00  0.0000  0.0000  0.0042  ...  0.0005 -0.0213  0.0102   
2022-11-30 01:00:00 -0.0000 -0.0000  0.0056  ... -0.0007 -0.0098  0.0011   
2022-11-30 01:30:00 -0.0002 -0.0001 -0.0001  ...  0.0004 -0.0119  0.0032   
2022-11-30 02:00:00 -0.0001 -0.0001 -0.0005  ... -0.0003 -0.0088  0.0135   
...                     ...     ...     ...  ...     ...     ...     ...   
2023-05-09 15:30:00  0.0000  0.0000  0.0000  ...  0.0112  0.0300  0.0101   
2023-05-09 16:00:00  0.0000  0.0000  0.0000  ...  0.0169  0.0159 -0.0012   
2023-05-09 16:30:00  0.0000  0.0000  0.0000  ...  0.0060  0.0447  0.0028   
2023-05-09 17:00:00  0.0000  0.0000  0.0000  ...  0.0126  0.0689 -0.0145   
2023-05-09 17:30:00  0.0000  0.0000  0.0000  ...  0.0069  0.0174 -0.0493   

height                                                                       
tower                            ue                      uw                  
measurement          w_h2o_  u_h2o_  v_h2o_  w_h2o_  u_h2o_  v_h2o_  w_h2o_  
time                                                                         
2022-11-30 00:00:00  0.0009 -0.0040  0.0051 -0.0003 -0.0298  0.0238  0.0001  
2022-11-30 00:30:00  0.0005 -0.0119  0.0123  0.0005 -0.0179  0.0177  0.0005  
2022-11-30 01:00:00 -0.0002 -0.0048  0.0034  0.0003 -0.0110  0.0028  0.0007  
2022-11-30 01:30:00  0.0006 -0.0108  0.0058 -0.0006 -0.0108  0.0062 -0.0002  
2022-11-30 02:00:00 -0.0004  0.0060  0.0244 -0.0002 -0.0002  0.0046 -0.0013  
...                     ...     ...     ...     ...     ...     ...     ...  
2023-05-09 15:30:00  0.0155  0.0509 -0.0096  0.0105  0.0506 -0.0205  0.0181  
2023-05-09 16:00:00  0.0085  0.0051  0.0032  0.0100 -0.0015  0.0053  0.0127  
2023-05-09 16:30:00  0.0067  0.0537 -0.0012  0.0023  0.0387 -0.0195  0.0086  
2023-05-09 17:00:00  0.0105  0.0600 -0.0091  0.0115  0.0698 -0.0284  0.0126  
2023-05-09 17:30:00  0.0064  0.0055 -0.0591  0.0059  0.0075 -0.0527  0.0059  

[7714 rows x 36 columns]

## Humidity measurements

In [19]:
abs_hum_field_df = tidy_df[tidy_df.measurement.isin(['Water vapor density']) & tidy_df.height.isin(heights)]
abs_hum_field_df.measurement = 'q'
abs_hum_field_df = round(
    abs_hum_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)
abs_hum_field_df

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/2134385205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abs_hum_field_df.measurement = 'q'


height                 1.0                             3.0                   \
tower                     c       d      ue      uw       c       d      ue   
measurement               q       q       q       q       q       q       q   
time                                                                          
2022-11-30 00:00:00  0.6355  1.1561  0.8129  0.6652  1.2019  0.7553  0.7909   
2022-11-30 00:30:00  0.7348  1.2510  0.8063  0.6783  1.3128  0.7867  0.7596   
2022-11-30 01:00:00  0.7671  1.2537  0.8071  0.6670  1.3324  0.7876  0.7876   
2022-11-30 01:30:00  0.7242  1.2071  0.8064  0.6342  1.3038  0.7969  0.8061   
2022-11-30 02:00:00  0.7210  1.2501  0.8075  0.6270  1.3360  0.6982  0.7069   
...                     ...     ...     ...     ...     ...     ...     ...   
2023-05-09 15:30:00  2.6024     NaN     NaN     NaN  2.4664  2.6617  2.5909   
2023-05-09 16:00:00  2.5730     NaN     NaN     NaN  2.4660  2.6428  2.5880   
2023-05-09 16:30:00  2.8056     NaN     NaN     NaN  2.6200  2.8197  2.7385   
2023-05-09 17:00:00  2.6395     NaN     NaN     NaN  2.5135  2.7082  2.6553   
2023-05-09 17:30:00  3.0323     NaN     NaN     NaN  2.7600  2.9733  2.8969   

height                         10.0                          
tower                    uw       c       d      ue      uw  
measurement               q       q       q       q       q  
time                                                         
2022-11-30 00:00:00  0.8705  1.0104  0.9129  1.0040  1.0092  
2022-11-30 00:30:00  0.9388  1.0503  0.9741  1.0798  1.1279  
2022-11-30 01:00:00  0.9670  1.0805  1.0060  1.1239  1.1660  
2022-11-30 01:30:00  0.9272  1.0612  0.9827  1.0905  1.1444  
2022-11-30 02:00:00  0.9474  1.0986  1.0048  1.1193  1.1829  
...                     ...     ...     ...     ...     ...  
2023-05-09 15:30:00  2.7364  2.1723  2.4516  2.5774  2.2098  
2023-05-09 16:00:00  2.7249  2.2089  2.4480  2.5768  2.2006  
2023-05-09 16:30:00  2.9030  2.3425  2.5744  2.6768  2.3313  
2023-05-09 17:00:00  2.7873  2.2988  2.4970  2.6286  2.2827  
2023-05-09 17:30:00  3.0886  2.4174  2.6028  2.7749  2.4034  

[7440 rows x 12 columns]

## Advective flux measurements

In [20]:
ls = []
for h in wind_field_df.columns.get_level_values('height').unique():
    for t in wind_field_df.columns.get_level_values('tower').unique():
        this_wind_df = wind_field_df[(h,t)].copy()
        this_abs_hum_df = abs_hum_field_df[(h,t)].copy()  
        this_wind_df['uq'] = this_wind_df['u']*this_abs_hum_df['q']
        this_wind_df['vq'] = this_wind_df['v']*this_abs_hum_df['q']
        this_wind_df['wq'] = this_wind_df['w']*this_abs_hum_df['q']
        new = pd.concat([this_wind_df], axis=1, keys=[(h,t)])
        ls.append(new.drop(columns=[(h,t,'u'),(h,t,'v'),(h,t,'w')]))

adv_flux_field_df = ls[0]
for l in ls[1:]:
    adv_flux_field_df = adv_flux_field_df.join(l)
adv_flux_field_df.columns = adv_flux_field_df.columns.set_names('height', level=0)
adv_flux_field_df.columns = adv_flux_field_df.columns.set_names('tower', level=1)
adv_flux_field_df.head()

height                   1.0                                           \
tower                       c                             d             
measurement                uq        vq        wq        uq        vq   
time                                                                    
2022-11-30 00:00:00  0.536489 -1.272207 -0.024657  0.952742 -2.152774   
2022-11-30 00:30:00 -0.190460 -0.754419 -0.018737 -0.214797 -1.283901   
2022-11-30 01:00:00  0.229746 -0.869201 -0.022323  0.437917 -1.346975   
2022-11-30 01:30:00  0.474279 -1.159010 -0.024478  0.776286 -1.774558   
2022-11-30 02:00:00 -0.209378 -0.625468 -0.021846 -0.308650 -1.116714   

height                                                                 ...  \
tower                                ue                            uw  ...   
measurement                wq        uq        vq        wq        uq  ...   
time                                                                   ...   
2022-11-30 00:00:00 -0.029827  0.619511 -1.574831 -0.039913  0.207077  ...   
2022-11-30 00:30:00 -0.037155 -0.261644 -0.881608 -0.041363 -0.158247  ...   
2022-11-30 01:00:00 -0.024949  0.231557 -0.909521 -0.038983  0.059029  ...   
2022-11-30 01:30:00 -0.027884  0.487308 -1.252984 -0.041046  0.082700  ...   
2022-11-30 02:00:00 -0.031378 -0.293123 -0.787555 -0.045139 -0.185216  ...   

height                   10.0                                          \
tower                       c         d                            ue   
measurement                wq        uq        vq        wq        uq   
time                                                                    
2022-11-30 00:00:00 -0.083156  2.301969 -1.998521 -0.005934  2.215125   
2022-11-30 00:30:00 -0.035920 -0.070914 -0.796132  0.018410 -0.273081   
2022-11-30 01:00:00 -0.061156  0.611346 -0.665972  0.002817  0.519916   
2022-11-30 01:30:00 -0.097843  1.865263 -1.478963 -0.031643  1.885584   
2022-11-30 02:00:00 -0.030871 -0.023110 -0.587507  0.023110 -0.201138   

height                                                                 
tower                                          uw                      
measurement                vq        wq        uq        vq        wq  
time                                                                   
2022-11-30 00:00:00 -2.381086  0.038654  1.923737 -2.736042 -0.000101  
2022-11-30 00:30:00 -0.832310  0.031098 -0.255808 -0.876829 -0.013648  
2022-11-30 01:00:00 -0.826516  0.025737  0.555249 -1.012088 -0.024486  
2022-11-30 01:30:00 -1.837929 -0.006434  1.671854 -2.292462 -0.085944  
2022-11-30 02:00:00 -0.536928  0.061114 -0.083749 -0.608957 -0.040455  

[5 rows x 36 columns]

# Divergence calculations

In [21]:
def divergence(f,sp):
    """ 
    From: https://stackoverflow.com/a/67971515
    Computes divergence of vector field 
    f: array -> vector field components [Fx,Fy,Fz,...]
    sp: array -> spacing between points in respecitve directions [spx, spy,spz,...]
    """
    num_dims = len(f)
    return np.ufunc.reduce(np.add, [np.gradient(f[i], sp[i], axis=i) for i in range(num_dims)])

## Calculate interpolated fields

In [22]:
# Initialize lists to store results
wind_fields = []
advectedflux_fields = []
abshumidity_fields = []
grid_spacings = []
turbulentflux_fields = []
timestamps = []

for i in wind_field_df.index:
    if ( # sometimes we don't have all measurements - this ensures we only retrieve data that exists
        (i in wind_field_df.index) and (i in adv_flux_field_df.index) and  
        (i in abs_hum_field_df.index) and  (i in turb_flux_field_df.index)
    ):
        # Isolate wind speed (u_i) measurements for this timestamp
        wind_field_vals = pd.DataFrame(
            wind_field_df.loc[i]
        ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
        wind_field_vals.columns = wind_field_vals.columns.droplevel(0)
        
        # Isolate advective flux (u_i*q) measurements for this timestamp
        adv_flux_field_vals = pd.DataFrame(
            adv_flux_field_df.loc[i]
        ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
        adv_flux_field_vals.columns = adv_flux_field_vals.columns.droplevel(0)

        # Isolate humidity (q) measurements for this timestamp
        abs_humidity_field_vals = pd.DataFrame(
            abs_hum_field_df.loc[i]
        ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
        abs_humidity_field_vals.columns = abs_humidity_field_vals.columns.droplevel(0)

        # Isolate turbulent flux (u_i'q') measurements for this timestamp
        turb_flux_field_vals = pd.DataFrame(
            turb_flux_field_df.loc[i]
        ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
        turb_flux_field_vals.columns = turb_flux_field_vals.columns.droplevel(0)

        # Combine all measurements of fields and instrument locations into one dataframe
        points_and_fields = instrument_loc_df.join(
            wind_field_vals, how='right' # join on right df, so we drop instruments that we don't have measurements for
        ).join(
            adv_flux_field_vals,
        ).join(
            abs_humidity_field_vals,
        ).join(
            turb_flux_field_vals,
        )

        # Create a meshgrid for the interpolation
        xx, yy, zz = np.meshgrid(
            np.linspace(points_and_fields.x.min(), points_and_fields.x.max(), HORIZ_GRID_SPACING),
            np.linspace(points_and_fields.y.min(), points_and_fields.y.max(), HORIZ_GRID_SPACING),
            np.linspace(points_and_fields.z.min(), points_and_fields.z.max(), VERT_GRID_SPACING)
        )
        points = np.transpose(np.vstack((points_and_fields.x, points_and_fields.y, points_and_fields.z)))

        # Interpolate wind field
        u_interp = interpolate.griddata(points, points_and_fields['u'], (xx, yy, zz), method='linear')
        v_interp = interpolate.griddata(points, points_and_fields['v'], (xx, yy, zz), method='linear')
        w_interp = interpolate.griddata(points, points_and_fields['w'], (xx, yy, zz), method='linear')

        # Interpolate advected flux field
        uq_interp = interpolate.griddata(points, points_and_fields['uq'], (xx, yy, zz), method='linear')
        vq_interp = interpolate.griddata(points, points_and_fields['vq'], (xx, yy, zz), method='linear')
        wq_interp = interpolate.griddata(points, points_and_fields['wq'], (xx, yy, zz), method='linear')

        # Interpolate abs. humidity field
        q_interp = interpolate.griddata(points, points_and_fields['q'], (xx, yy, zz), method='linear')

        # Interpolate turbulent flux field
        u_q__interp = interpolate.griddata(points, points_and_fields['u_h2o_'], (xx, yy, zz), method='linear')
        v_q__interp = interpolate.griddata(points, points_and_fields['v_h2o_'], (xx, yy, zz), method='linear')
        w_q__interp = interpolate.griddata(points, points_and_fields['w_h2o_'], (xx, yy, zz), method='linear')

        # Combine interpolated components into vector fields
        F = np.array([u_interp, v_interp, w_interp])
        Fq = np.array([uq_interp, vq_interp, wq_interp])
        F_q_ = np.array([u_q__interp, v_q__interp, w_q__interp])

        # Record grid spacing        
        sp_x = np.diff(xx[0,:,0]).mean()
        sp_y = np.diff(yy[:,0,0]).mean()
        sp_z = np.diff(zz[0,0,:]).mean()
        sp = [sp_x, sp_y, sp_z]

        # Append interpolated fields to our results lists
        wind_fields.append(F)
        advectedflux_fields.append(Fq)
        abshumidity_fields.append(q_interp)
        turbulentflux_fields.append(F_q_)
        grid_spacings.append(sp)
        timestamps.append(i)

## Calculate divergences


In [23]:
# Initialize lists to store results
div_abshumidity_field_ls = []

div_wind_field_ls = []
div_wind_field_lateral_ls = []
div_wind_field_vertical_ls = []

div_turbulentflux_field_ls = []
div_turbulentflux_field_lateral_ls = []
div_turbulentflux_field_vertical_ls = []

div_advectedflux_field_ls = []
div_advectedflux_field_lateral_ls = []
div_advectedflux_field_vertical_ls = []

div_advectedflux_field_scalargradient_ls = []
div_advectedflux_field_scalargradient_lateral_ls = []
div_advectedflux_field_scalargradient_vertical_ls = []

div_advectedflux_field_windgradient_ls = []
div_advectedflux_field_windgradient_lateral_ls = []
div_advectedflux_field_windgradient_vertical_ls = []

# iterate over each field (timestamp) and calculate divergences
for i in range(0, len(wind_fields)):
    wind_field          = wind_fields[i]
    advectedflux_field  = advectedflux_fields[i]
    abshumidity_field   = abshumidity_fields[i]
    turbulentflux_field = turbulentflux_fields[i]
    grid_spacing        = grid_spacings[i]

    # create wind fields with just lateral and vertical components
    wind_field_vertical_only = wind_field.copy()
    wind_field_lateral_only  = wind_field.copy()
    wind_field_vertical_only[0] = 0
    wind_field_vertical_only[1] = 0
    wind_field_lateral_only[2] = 0

    advectedflux_field_vertical_only = advectedflux_field.copy()
    advectedflux_field_lateral_only  = advectedflux_field.copy()
    advectedflux_field_vertical_only[0] = 0
    advectedflux_field_vertical_only[1] = 0
    advectedflux_field_lateral_only[2] = 0

    turbulentflux_field_vertical_only = turbulentflux_field.copy()
    turbulentflux_field_lateral_only = turbulentflux_field.copy()
    turbulentflux_field_vertical_only[0] = 0
    turbulentflux_field_vertical_only[1] = 0
    turbulentflux_field_lateral_only[2] = 0

    # calculate divergences

    # For humidity field
    div_abshumidity_field    = np.gradient(
        abshumidity_field,  grid_spacing[0],    axis=0
    ) + np.gradient(
        abshumidity_field,  grid_spacing[1],    axis=1
    ) + np.gradient(
        abshumidity_field,  grid_spacing[2],    axis=2
    )

    # For wind field
    div_wind_field          = divergence(wind_field, grid_spacing)
    div_wind_field_lateral  = divergence(wind_field_lateral_only, grid_spacing)
    div_wind_field_vertical  = divergence(wind_field_vertical_only, grid_spacing)

    # For turbulent flux
    div_turbulentflux_field = divergence(turbulentflux_field, grid_spacing)
    div_turbulentflux_field_lateral = divergence(turbulentflux_field_lateral_only, grid_spacing)
    div_turbulentflux_field_vertical = divergence(turbulentflux_field_vertical_only, grid_spacing)

    # For advected flux of form: d/dx (u_i q)
    div_advectedflux_field  = divergence(advectedflux_field, grid_spacing)
    div_advectedflux_field_lateral  = divergence(advectedflux_field_lateral_only, grid_spacing)
    div_advectedflux_field_vertical = divergence(advectedflux_field_vertical_only, grid_spacing)
    
    # For advected flux of form: u d/dx (q)
    div_advectedflux_field_scalargradient   = wind_field*div_abshumidity_field
    div_advectedflux_field_scalargradient_lateral   = wind_field_lateral_only*div_abshumidity_field
    div_advectedflux_field_scalargradient_vertical   = wind_field_vertical_only*div_abshumidity_field
    
    # For advected flux of form: q d/dx (u)
    div_advectedflux_field_windgradient         = abshumidity_field*div_wind_field
    div_advectedflux_field_windgradient_lateral = abshumidity_field*div_wind_field_lateral
    div_advectedflux_field_windgradient_vertical= abshumidity_field*div_wind_field_vertical

    # Divergence of wind field
    ##########################################
    div_wind_field_ls.append(
        div_wind_field
    )
    div_wind_field_lateral_ls.append(
        div_wind_field_lateral
    )
    div_wind_field_vertical_ls.append(
        div_wind_field_vertical
    )
    # Divergence of advected flux
    ##########################################
    div_advectedflux_field_ls.append(
        div_advectedflux_field
    )
    div_advectedflux_field_lateral_ls.append(
        div_advectedflux_field_lateral
    )
    div_advectedflux_field_vertical_ls.append(
        div_advectedflux_field_vertical
    )
    # Divergence of turbulent flux
    ##########################################
    div_turbulentflux_field_ls.append(
        div_turbulentflux_field
    )
    div_turbulentflux_field_lateral_ls.append(
        div_turbulentflux_field_lateral
    )
    div_turbulentflux_field_vertical_ls.append(
        div_turbulentflux_field_vertical
    )
    # Divergence of abs humidity field
    ##########################################
    div_abshumidity_field_ls.append(
        div_abshumidity_field
    )
    # Divergence of advected field, using scalar gradient
    ##########################################
    div_advectedflux_field_scalargradient_ls.append(
        div_advectedflux_field_scalargradient
    )
    div_advectedflux_field_scalargradient_lateral_ls.append(
        div_advectedflux_field_scalargradient_lateral
    )
    div_advectedflux_field_scalargradient_vertical_ls.append(
        div_advectedflux_field_scalargradient_vertical
    )
    # Divergence of advected field, using wind gradient
    ##########################################
    div_advectedflux_field_windgradient_ls.append(
        div_advectedflux_field_windgradient
    )
    div_advectedflux_field_windgradient_lateral_ls.append(
        div_advectedflux_field_windgradient_lateral
    )
    div_advectedflux_field_windgradient_vertical_ls.append(
        div_advectedflux_field_windgradient_vertical
    )

## Calculate spatially averaged divergence values

In [24]:
conservation_spatial_mean_df = pd.DataFrame({
    'div_wind_field' : [
        np.nanmean(item) for item in div_wind_field_ls
    ],
    'div_wind_field_lateral' : [
        np.nanmean(item) for item in div_wind_field_lateral_ls
    ],
    'div_wind_field_vertical' : [
        np.nanmean(item) for item in div_wind_field_vertical_ls
    ],
    'div_advectedflux_field' : [
        np.nanmean(item) for item in div_advectedflux_field_ls
    ],
    'div_advectedflux_field_lateral' : [
        np.nanmean(item) for item in div_advectedflux_field_lateral_ls
    ],
    'div_advectedflux_field_vertical' : [
        np.nanmean(item) for item in div_advectedflux_field_vertical_ls
    ],
    'div_turbulentflux_field' : [
        np.nanmean(item) for item in div_turbulentflux_field_ls
    ],
    'div_turbulentflux_field_lateral' : [
        np.nanmean(item) for item in div_turbulentflux_field_lateral_ls
    ],
    'div_turbulentflux_field_vertical' : [
        np.nanmean(item) for item in div_turbulentflux_field_vertical_ls
    ],
    'div_abshumidity_field' : [
        np.nanmean(item) for item in div_abshumidity_field_ls
    ],
    'div_advectedflux_field_scalargradient' : [
        np.nanmean(item) for item in div_advectedflux_field_scalargradient_ls
    ],
    'div_advectedflux_field_scalargradient_lateral' : [
        np.nanmean(item) for item in div_advectedflux_field_scalargradient_lateral_ls
    ],
    'div_advectedflux_field_scalargradient_vertical' : [
        np.nanmean(item) for item in div_advectedflux_field_scalargradient_vertical_ls
    ],
    'div_advectedflux_field_windgradient' : [
        np.nanmean(item) for item in div_advectedflux_field_windgradient_ls
    ],
    'div_advectedflux_field_windgradient_lateral' : [
        np.nanmean(item) for item in div_advectedflux_field_windgradient_lateral_ls
    ],
    'div_advectedflux_field_windgradient_vertical' : [
        np.nanmean(item) for item in div_advectedflux_field_windgradient_vertical_ls
    ],
    'abs_humidity_field': [
        np.nanmean(item) for item in abshumidity_fields
    ]
}, index = timestamps)

conservation_spatial_mean_df[
    'div_advectedflux_field_summed'
] = conservation_spatial_mean_df['div_advectedflux_field'] -\
    conservation_spatial_mean_df['div_advectedflux_field_windgradient']

conservation_spatial_mean_df[
    'div_advectedflux_field_summed_lateral'
] = conservation_spatial_mean_df['div_advectedflux_field_lateral'] -\
    conservation_spatial_mean_df['div_advectedflux_field_windgradient_lateral']

conservation_spatial_mean_df[
    'div_advectedflux_field_summed_vertical'
] = conservation_spatial_mean_df['div_advectedflux_field_vertical'] -\
    conservation_spatial_mean_df['div_advectedflux_field_windgradient_vertical']

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1636442532.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(item) for item in div_wind_field_ls
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1636442532.py:6: RuntimeWarning: Mean of empty slice
  np.nanmean(item) for item in div_wind_field_lateral_ls
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1636442532.py:9: RuntimeWarning: Mean of empty slice
  np.nanmean(item) for item in div_wind_field_vertical_ls
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1636442532.py:12: RuntimeWarning: Mean of empty slice
  np.nanmean(item) for item in div_advectedflux_field_ls
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1636442532.py:15: RuntimeWarning: Mean of empty slice
  np.nanmean(item) for item in div_advectedflux_field_lateral_ls
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/1636442532.py:18: RuntimeWarning: Mean of empty slice
  n

## Decide which advection calculation we trust

In [25]:
src = conservation_spatial_mean_df.groupby(
    conservation_spatial_mean_df.index.floor('60T').time
).mean()
src = src.assign(time = src.index.to_series().apply(lambda t: dt.datetime.combine(dt.date(2011, 1, 1), t)))
alt.Chart(src).transform_fold([
    'div_advectedflux_field',
    'div_advectedflux_field_scalargradient',
    'div_advectedflux_field_windgradient',
    # 'div_advectedflux_field_summed',
]).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('value:Q'),
    alt.Color('key:N')
).properties(height = 200, width = 300).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91994/2004861624.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  conservation_spatial_mean_df.index.floor('60T').time


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [26]:
conservation_spatial_mean_df = conservation_spatial_mean_df.drop(columns=[
    'div_advectedflux_field',
    'div_advectedflux_field_lateral',  
    'div_advectedflux_field_vertical',
    'div_advectedflux_field_summed',
    'div_advectedflux_field_windgradient',
    'div_advectedflux_field_summed_lateral',
    'div_advectedflux_field_windgradient_lateral',  
    'div_advectedflux_field_summed_vertical',
    'div_advectedflux_field_windgradient_vertical',
])

conservation_spatial_mean_df = conservation_spatial_mean_df.rename(columns={
    'div_advectedflux_field_scalargradient' : 'div_advectedflux_field',
    'div_advectedflux_field_scalargradient_lateral' : 'div_advectedflux_field_lateral',  
    'div_advectedflux_field_scalargradient_vertical' : 'div_advectedflux_field_vertical',
})

In [27]:
line = alt.Chart().mark_rule().encode(y=alt.datum(0))

def conservation_spatial_mean_composite_plot(columns, title, times_filter = None, normalize = None):
    if times_filter is not None:
        src = conservation_spatial_mean_df[conservation_spatial_mean_df.index.isin(times_filter)]
    else:
        src = conservation_spatial_mean_df
    if normalize is not None:
        src = src * normalize
    return line + alt.Chart(
        src.reset_index()
    ).transform_fold(
        columns
    ).mark_line().encode(
        alt.X('hoursminutes(index):T').title('time'),
        alt.Y('median(value):Q'),
        alt.Color('key:N')
    ).properties(height = 200, width = 200, title=title)

In [28]:
alt.Chart(
        conservation_spatial_mean_df[[
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ]].reset_index()
    ).transform_fold(
        [
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ]
    ).mark_line().encode(
        alt.X('hoursminutes(index):T').title('time'),
        alt.Y('median(value):Q'),
        alt.Color('key:N')
    ).properties(height = 200, width = 200, )


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [29]:
(
    conservation_spatial_mean_composite_plot([
            'div_wind_field',
            'div_wind_field_lateral',
            'div_wind_field_vertical',
        ], 'Divergence of wind field (1/s)'
    ) | (
        conservation_spatial_mean_composite_plot([
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ], 'Divergence of advective flux (g/m^3)'
        ) | conservation_spatial_mean_composite_plot([
                'div_turbulentflux_field',
                'div_turbulentflux_field_lateral',
                'div_turbulentflux_field_vertical',
            ], 'Divergence of turbulent flux (g/m^3)'
        )
    ).resolve_scale(color='independent', x='shared', y='shared')
).resolve_scale(
    color='independent', x='shared', y='independent',
).configure_legend(orient='top', columns=1).properties(
    title='All data'
).display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [30]:
(
    conservation_spatial_mean_composite_plot([
            'div_wind_field',
            'div_wind_field_lateral',
            'div_wind_field_vertical',
        ], 'Divergence of wind field (1/s)',
        set(nobs_times).intersection(set(is_not_snowing_dates))
    ) | (
        conservation_spatial_mean_composite_plot([
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ], 'Divergence of advective flux (g/m^3)',
        set(nobs_times).intersection(set(is_not_snowing_dates))
        ) | conservation_spatial_mean_composite_plot([
                'div_turbulentflux_field',
                'div_turbulentflux_field_lateral',
                'div_turbulentflux_field_vertical',
            ], 'Divergence of turbulent flux (g/m^3)',
        set(nobs_times).intersection(set(is_not_snowing_dates))
        )
    ).resolve_scale(color='independent', x='shared', y='shared')
).resolve_scale(
    color='independent', x='shared', y='independent',
).configure_legend(orient='top', columns=1).properties(
    title='All data'
).display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [31]:
src = tidy_df[
    tidy_df.time.isin(
        set(nobs_times).intersection(set(is_not_snowing_dates))
    )
]
src = src[src.variable.isin(['w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c'])]
ec_flux_chart = alt.Chart(
    src
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('height:O')
).properties(height = 200, width = 200, title='Vertical turbulent flux (g/m^2/s)')

In [32]:
(
    conservation_spatial_mean_composite_plot([
            'div_wind_field',
            'div_wind_field_lateral',
            'div_wind_field_vertical',
        ], 'Divergence of wind field (1/s)',
        set(nobs_times).intersection(set(is_not_snowing_dates)),
        normalize=7
    ) | (
        conservation_spatial_mean_composite_plot([
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ], 'Divergence of advective flux (g/m^2/s)',
        set(nobs_times).intersection(set(is_not_snowing_dates)),
        normalize=7
        ) | conservation_spatial_mean_composite_plot([
                'div_turbulentflux_field',
                'div_turbulentflux_field_lateral',
                'div_turbulentflux_field_vertical',
            ], 'Divergence of turbulent flux (g/m^2/s)',
        set(nobs_times).intersection(set(is_not_snowing_dates)),
        normalize=7
        )
        |
        ec_flux_chart
    ).resolve_scale(color='independent', x='shared', y='shared')
).resolve_scale(
    color='independent', x='shared', y='independent',
).configure_legend(orient='top', columns=1).properties(
    title='All data'
).display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [33]:
(
    conservation_spatial_mean_composite_plot([
            'div_wind_field',
            'div_wind_field_lateral',
            'div_wind_field_vertical',
        ], 'Divergence of wind field (1/s)',
        set(bs_times),
        normalize=7
    ) | (
        conservation_spatial_mean_composite_plot([
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ], 'Divergence of advective flux (g/m^2/s)',
        set(bs_times),
        normalize=7
        ) | conservation_spatial_mean_composite_plot([
                'div_turbulentflux_field',
                'div_turbulentflux_field_lateral',
                'div_turbulentflux_field_vertical',
            ], 'Divergence of turbulent flux (g/m^2/s)',
        set(bs_times),
        normalize=7
        )
        |
        ec_flux_chart
    ).resolve_scale(color='independent', x='shared', y='shared')
).resolve_scale(
    color='independent', x='shared', y='independent',
).configure_legend(orient='top', columns=1).properties(
    title='All data'
).display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [34]:
alt.Chart(
    tidy_df.query("measurement == 'air density'").query("height > 0")
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('height:O')
) | alt.Chart(
    tidy_df.query("measurement == 'temperature'").query("height > 0")
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('height:O')
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting
